# 类的方法

## 方法（Method）
---

In [1]:
class MyClass(object):
    i = 1
    def foo(self):
        pass
mc = MyClass()
mc.foo()

上面定义了一个类`MyClass`，类中定义了一个方法`foo`,如果像函数一样对`foo`自身的调用都会失败：

In [2]:
foo()

NameError: name 'foo' is not defined

直接调用引发了NameError的异常，因为在全局函数中没有这样的函数存在。也就是说`foo`是一个方法表示它属于一个类而不是全局空间中的名字。

In [3]:
MyClass.foo()

TypeError: foo() missing 1 required positional argument: 'self'

In [4]:
MyClass.i

1

  直接调用`MyClass.foo()`会抛出TypeError的异常，这是因为为了与OOP惯例保持一致，Python严格要求：没有实例的方法是不能被调用的。这种限制即Python所描述的绑定概念（binding），在此，**方法必须绑定（到一个实例）才能直接被调用**。

## 绑定和方法调用
---

+ 方法仅仅是类内部定义的函数（这意味着方法是类属性而不是实例属性）。
+ 方法只有在其所属的类拥有实例时才能够被调用，当存在一个实例时，方法才被认为是绑定到那个实例了，否则就是未绑定的。
+ 任何一个方法定义中的第一个参数都是`self`，他表示调用此方法的实例对象

### 对象的绑定方法

  首先看一个例子<sup>[1]</sup>:

In [5]:
class People:
    def __init__(self,name,age):
        self.name = name
        self.age = age
    def talk(self):
        print(self.__dict__)

A = People('xiaoming',18)
print(A.talk)

<bound method People.talk of <__main__.People object at 0x0000019481B9A438>>


从上面的输出结果来看，`talk()`这个类中的方法，是绑定给对象使用的。下面，再看看另外一种情况：

In [6]:
class People:
    def __init__(self,name,age):
        self.name = name
        self.age = age
    def talk():
        pass
    
B = People('xiaohong',16)
print(B.talk)
print(People.talk)

<bound method People.talk of <__main__.People object at 0x0000019481B9AF98>>
<function People.talk at 0x0000019481BA1400>


 现在，我们将`talk()`函数的`self`变量去掉，使其为类中的函数，`B.talk`仍然是bound method。这说明，不管是类中的方法，还是类中函数，默认情况下都是绑定给对象使用的。绑定给对象使用有一种好处，那就是不用手动将对象传入。对象是自动传到类中。而如果类来调用类中的方法`People.talk`，那么这个方法仅仅只是一个函数，那么既然是函数，就不会有自动传值这一功能。

 再重新看前面两个例子

In [7]:
class People:
    def __init__(self,name,age):
        self.name = name
        self.age = age
    def talk(self):
        print(self.__class__)

A = People('xiaoming',18)
A.talk()
People.talk()

<class '__main__.People'>


TypeError: talk() missing 1 required positional argument: 'self'

`A.talk()`运行正常，`People.talk()`报错的原因正式前文所述的**没有实例的方法是不能被调用的**,如果想用类名调用类中的方法时，方法有几个参数就要手动传几个参数进去：

In [8]:
People.talk(1234)

<class 'int'>


同样，对于类中定义一个函数的情况，实例化的对象调用该函数也会报错:

In [9]:
class People:
    def __init__(self,name,age):
        self.name = name
        self.age = age
    def talk():
        print('say something')
    
B = People('xiaohong',16)
People.talk()  #正常执行
B.talk()       #报错：talk() takes 0 positional arguments but 1 was given

say something


TypeError: talk() takes 0 positional arguments but 1 was given

### 类的绑定方法

上面的例子指出，调用类中的方法都要通过实例化后在实例中运行，没有实例的方法是不能被调用的。因此，如果我想通过类中的方法初始化某个变量，就会面临一些问题。比如上述例子中使用`A = People.talk()`去初始化`A`就会给出`TypeError: talk() missing 1 required positional argument: 'self'`的错误。那么有什么办法能让类的方法不在实例中运行吗？

让我们假设一个处理时间的类<sup>[5]</sup>:
```Python
class Date(object):

    def __init__(self, day=0, month=0, year=0):
        self.day = day
        self.month = month
        self.year = year
```

类中，我们用`__init__`作为初始化实例的方式并用`self`作为第一个参数引用所创建的实例。

#####  Class Method

假设我们得到了给定格式的('dd-mm-yyyy')时间信息字符串，并依此创建大量`Date`实例，我们需要做如下两个步骤：
    1. 将字符串按照天/月/年的格式解析为三个int变量或者一个三元素的tuple变量
    2. 用解析好的变量实例化`Date`
对于第一步可以通过代码实现：
```Python
day, month, year = map(int, string_date.split('-'))
date1 = Date(day, month, year)
```
那我能不能把这段代码放入到类中呢？这时就要用到`classmethod`:
```Python
    @classmethod
    def from_string(cls, date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        date1 = cls(day, month, year)
        return date1

date2 = Date.from_string('11-09-2012')
```
完整的代码如下：

In [10]:
class Date(object):
    def __init__(self, day=0, month=0, year=0):
        self.day = day
        self.month = month
        self.year = year
        
    @classmethod
    def from_string(cls, date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        date1 = cls(day, month, year)
        return date1
    
    def printd(self):
        print(self.day)
        print(self.month)
        print(self.year)

date2 = Date.from_string('11-09-2012')
date2.printd()

11
9
2012


#####  Static method

`staticmethod`与`classmethod`用法相似，不同的地方在于`classmethod`的第一个参数必须用于引用类对象(`self`)，而`staticmethod`甚至可以没有任何参数。

比如我们要验证给定的字符串是否是有效的日期：
```Python
    @staticmethod
    def is_date_valid(date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        return day <= 31 and month <= 12 and year <= 3999

    # usage:
    is_date = Date.is_date_valid('11-09-2012')
```
从这个例子中可以看出，`staticmethod`其仅仅只是一个函数。

下面给出一个直观的例子比较`staticmethod`和`classmethod`<sup>[4]</sup>
```Python
class Kls(object):
    def __init__(self, data):
        self.data = data
    def printd(self):
        print(self.data)
    @staticmethod
    def smethod(*arg):
        print('Static:', arg)
    @classmethod
    def cmethod(*arg):
        print('Class:', arg)

>>> ik = Kls(23)
>>> ik.printd()
23
>>> ik.smethod()
Static: ()
>>> ik.cmethod()
Class: (<class '__main__.Kls'>,)
>>> Kls.printd()
TypeError: unbound method printd() must be called with Kls instance as first argument (got nothing instead)
>>> Kls.smethod()
Static: ()
>>> Kls.cmethod()
Class: (<class '__main__.Kls'>,)
```

#### Reference

1. https://www.cnblogs.com/vipchenwei/p/7126772.html
2. https://docs.python.org/3/library/functions.html?highlight=classmethod#staticmethod
3. https://docs.python.org/3/library/functions.html?highlight=classmethod#classmethod
4. http://www.wklken.me/posts/2013/12/22/difference-between-staticmethod-and-classmethod-in-python.html
5. https://stackoverflow.com/questions/12179271/meaning-of-classmethod-and-staticmethod-for-beginner/12179325#12179325